### naver news crawling - df에 담기

In [26]:

from bs4 import BeautifulSoup as bs
import json
import re
import sys
import time, random
import pandas as pd
import requests

def get_news(n_url):
    news_detail = []
    print(n_url)
    breq = requests.get(n_url)
    bsoup = bs(breq.content, 'html.parser')
    
    # 제목 파싱
    title = bsoup.select('h3#articleTitle')[0].text
    news_detail.append(title)
    
    # 날짜
    pdate = bsoup.select('.t11')[0].get_text()[:10]
    news_detail.append(pdate)
    
    # news text
    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', ' ')
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
    
    # 신문사 
    pcompany = bsoup.select("#footer address")[0].a.get_text()
    news_detail.append(pcompany)
    
    return news_detail

columns = ["날짜", "신문사", "제목", "내용"]
df = pd.DataFrame(columns=columns)

query = '코로나'   # url 인코딩 에러는 encoding parse.quote(query)
s_date = "2020.04.01"
e_date = "2020.04.08"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

while True:
    time.sleep(random.sample(range(3), 1)[0])
    print(page)
    url = f"https://search.naver.com/search.naver?where=news&query={query}&sort=1&field=1&ds={s_date}&de={e_date}&nso=so%3Ar%2Cp%3Afrom{s_from}to{e_to}%2Ca%3A&start={str(page)}"
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    
    req  = requests.get(url, headers=header)
    print (url)
    cont = req.content
    soup = bs(cont, 'html.parser')
    
    if soup.findAll("a", {"class" : "_sp_each_url"}) == []:
        break
        
    for urls in soup.findAll("a", {"class" : "_sp_each_url"}):
        try:
            if urls.attrs['href'].startswith("https://news.naver.com"):
                print(urls.attrs["href"])
                news_detail = get_news(urls.attrs["href"])
                df = df.append(pd.DataFrame(
                    [[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]], columns=columns))
        except Exception as e:
            print (e)
            continue
    page += 10
    if (page == 31):
        break;
            
    

1
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.04.01&de=2020.04.08&nso=so%3Ar%2Cp%3Afrom20200401to20200408%2Ca%3A&start=1
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533947
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533947
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=215&aid=0000862824
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=215&aid=0000862824
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533946
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533946
11
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.04.01&de=2020.04.08&nso=so%3Ar%2Cp%3Afrom20200401to20200408%2Ca%3A&start=11
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533946
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=001

### 임의적으로 담은 데이터 확인하기.

In [27]:
print (df)

           날짜     신문사                                         제목  \
0  2020.04.08    연합뉴스  러시아 코로나19 신규 확진자 이틀째 1천명대…"전체 8천672명"(종합)   
0  2020.04.08  한국경제TV            뉴욕증시, 코로나19 진정 기대 속 숨고르기…장중 강보합   
0  2020.04.08    연합뉴스                   이탈리아, 코로나19에 난민구조선 입항 금지   
0  2020.04.08    연합뉴스                   이탈리아, 코로나19에 난민구조선 입항 금지   
0  2020.04.08     뉴시스    日 코로나19 신규감염 긴급사태 익일 500명 돌파...누계 5673명   
0  2020.04.08    연합뉴스           WTO "코로나19로 올해 세계 무역 32% 감소할 수도"   
0  2020.04.08    연합뉴스           뉴욕증시 '코로나19 추이' 주목 속 숨고르기…장중 강보합   

                                                  내용  
0  "모스크바 발병자 5천841명"…현지 전문가 "6월중순께나 수그러들 것"(모스크바=...  
0  [한국경제TV 김주리 기자]미국 뉴욕증시가 소폭 오름세를 보이고 있다.8일(현지시간...  
0  마스크를 쓴 채 난민 구조활동을 하는 독일 구호단체 '씨아이' 소속 요원들 모습. ...  
0  마스크를 쓴 채 난민 구조활동을 하는 독일 구호단체 '씨아이' 소속 요원들 모습. ...  
0  도쿄 144명 등 503명 폭증...총 114명 사망[후쿠오카=AP/뉴시스]8일 오...  
0  세계 무역에 미치는 코로나19 영향 전망[도표=WTO 제공](제네바=연합뉴스) 임은...  
0  유럽증시는 1%대 하락세(뉴욕=연합뉴스) 이준서 특파원 = 미국 뉴욕증시가 소폭 오...  


### ec2 mysql 연동해서 data 를 insert하기 - placeholder 구문을 사용해서 특수처리해서 하기!

In [5]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
import pandas as pd
import pymysql

# 접속
db = pymysql.connect(host='54.80.215.152',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')

# 커서 가져오기
cursor = db.cursor()

def get_news(n_url):
    news_detail = []
    print(n_url)
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    # 제목 파싱
    title = bsoup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜
    pdate = bsoup.select('.t11')[0].get_text()[:10]
    news_detail.append(pdate)

    # news text
    _text = bsoup.select('#articleBodyContents')[
        0].get_text().replace('\n', " ")
    btext = _text.replace(
        "// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())

    # 신문사
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail

columns = ['날짜', '신문사', '제목', '내용']
df = pd.DataFrame(columns=columns)

query = '코로나'   # url 인코딩 에러는 encoding parse.quote(query)
s_date = "2020.04.01"
e_date = "2020.04.08"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

while True:
    time.sleep(random.sample(range(3), 1)[0])
    print(page)

    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + \
        "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + \
        e_to + "%2Ca%3A&start=" + str(page)

    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    req = requests.get(url, headers=header)
    print(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    if soup.findAll("a", {"class": "_sp_each_url"}) == []:
        break

    for urls in soup.findAll("a", {"class": "_sp_each_url"}):
        try:
            if urls.attrs["href"].startswith("https://news.naver.com"):
                print(urls.attrs["href"])
                news_detail = get_news(urls.attrs["href"])
#      
                sql = """
                    INSERT INTO `news` (`date`, `company`, `title`, `content`) VALUES(%s, %s, %s, %s);
                """
#                 print(sql)
                # 날짜, 신문사, 제목, 내용
                cursor.execute(sql, (news_detail[1], news_detail[3], news_detail[0], news_detail[2]))
    
                db.commit()
        except Exception as e:
            print(e)
            continue
    page += 10
    
#     20번 돌리고 일단 stop
    if page == 211:
        break
db.close()

1
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.04.01&de=2020.04.08&nso=so%3Ar%2Cp%3Afrom20200401to20200408%2Ca%3A&start=1
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533947
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533947
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=215&aid=0000862824
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=215&aid=0000862824
11
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.04.01&de=2020.04.08&nso=so%3Ar%2Cp%3Afrom20200401to20200408%2Ca%3A&start=11
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009804632
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009804632
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011533945
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=001

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=018&aid=0004616101
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=018&aid=0004616101
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009804623
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009804623
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009804622
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009804622
141
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.04.01&de=2020.04.08&nso=so%3Ar%2Cp%3Afrom20200401to20200408%2Ca%3A&start=141
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=052&aid=0001425780
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=052&aid=0001425780
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=056&aid=0010816671
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&

In [45]:
print(df)

            날짜     신문사                                         제목  \
0  2020.04.08.    연합뉴스  러시아 코로나19 신규 확진자 이틀째 1천명대…"전체 8천672명"(종합)   
0  2020.04.08.  한국경제TV            뉴욕증시, 코로나19 진정 기대 속 숨고르기…장중 강보합   
0  2020.04.08.     뉴시스    日 코로나19 신규감염 긴급사태 익일 500명 돌파...누계 5673명   

                                                  내용  
0  "모스크바 발병자 5천841명"…현지 전문가 "6월중순께나 수그러들 것"(모스크바=...  
0  [한국경제TV 김주리 기자]미국 뉴욕증시가 소폭 오름세를 보이고 있다.8일(현지시간...  
0  도쿄 144명 등 503명 폭증...총 114명 사망[후쿠오카=AP/뉴시스]8일 오...  


### 1. mysql connect 연결하고, CREATE TABLE news 생성 

#### 잔재미 코딩 사이트 => https://www.fun-coding.org/mysql_basic6.html

In [1]:
import pymysql

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect(host='54.80.215.152',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')


In [3]:
import pymysql

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect(host='54.80.215.152',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')

# connection 객체로부터 cursr()메서드를 호출하여 Cursor 객체를 가져온다.
cursor = db.cursor()

# SQL 문 만들기
# int unsigned 는 id 같은 경우에는 음수를 사용하지 않으므로 그 범위가 양수로 옮겨간다. 굿
sql = '''
   CREATE TABLE news(
       id INT UNSIGNED NOT NULL AUTO_INCREMENT,
       date VARCHAR(25) NOT NULL,
       company VARCHAR(15) NOT NULL,
       title VARCHAR(100) NOT NULL,
       content VARCHAR(10000) NOT NULL,
       PRIMARY KEY(id) 
);  
'''

# 실행하기.
cursor.execute(sql)
cursor.execute('desc news')
results = cursor.fetchall()

for result in results:
    print (result)

# db에 complete 하기.
db.commit()

# db 연결 닫기
db.close()

('id', 'int(10) unsigned', 'NO', 'PRI', None, 'auto_increment')
('date', 'varchar(25)', 'NO', '', None, '')
('company', 'varchar(15)', 'NO', '', None, '')
('title', 'varchar(100)', 'NO', '', None, '')
('content', 'varchar(10000)', 'NO', '', None, '')


In [65]:
import pymysql
import pandas as pd

columns = ['인덱스','날짜', '신문사', '제목', '내용']
df = pd.DataFrame(columns=columns)


# df = df.append(pd.DataFrame(
#     [[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]], columns=columns))

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect(host='54.84.72.175',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')

# connection 객체로부터 cursr()메서드를 호출하여 Cursor 객체를 가져온다.
cursor = db.cursor()

sql = """
    SELECT * FROM news
"""

# 실행하기.
# cursor.execute(sql)
cursor.execute(sql)
results = cursor.fetchall()

for result in results:
    print (result[0], result[1], result[2], result[3], result[4])
    df = df.append(pd.DataFrame([[result[0], result[1], result[2], result[3], result[4]]], columns=columns))

# db에 complete 하기.
db.commit()

# db 연결 닫기
db.close()
print(df)

1 2020.04.08. 연합뉴스 러시아 코로나19 신규 확진자 이틀째 1천명대…"전체 8천672명"(종합) "모스크바 발병자 5천841명"…현지 전문가 "6월중순께나 수그러들 것"(모스크바=연합뉴스) 유철종 특파원 = 러시아의 신종 코로나바이러스 감염증(코로나19) 확진자 수가 하루 1천명대 이상의 증가세를 보이면서 누적 확진자가 8천600명을 넘어섰다.     타스 통신 등에 따르면 러시아 정부의 코로나19 유입 및 확산방지 대책본부는 8일(현지시간) "지난 하루 동안 모스크바를 포함한 56개 지역에서 1천175명의 추가 확진자가 나왔다"면서 "전체 누적 확진자가 8천672명(81개 지역)으로 늘었다"고 밝혔다.     수도 모스크바에선 660명의 추가 확진자가 나오면서 전체 감염자가 5천841명으로 증가했다.    모스크바시 대책본부는 "관내 신규 확진자 가운데 85%는 65세 이하"라며 "18세~45세 청장년층 비율이 40%로 가장 많다"고 전했다. 러시아 의료진. [리아노보스티=연합뉴스 자료사진]    이밖에 모스크바 외곽 모스크바주에서 95명, 제2도시 상트페테르부르크에서 34명, 인근 레닌그라드주에서 32명의 신규 확진자가 보고됐다.     코로나19로 인한 사망자도 하루 사이 5명이 추가되면서 모두 63명으로 늘어났다.     정부 대책본부는 지금까지 확진자 가운데 580명이 완치돼 퇴원했으며, 전체 검진 검사 건수는 90만1천 건이라고 전했다.     러시아의 코로나19 하루 신규 확진자는 전날(1천154명) 처음으로 1천명대를 넘어선 뒤 더 늘어가고 있다.     모스크바 교외 노보오가료보 관저에 머물고 있는 블라디미르 푸틴 대통령은 이날 지방정부 수장들과의 코로나19 대책 화상 회의를 주재하면서 전염병과 사투를 벌이고 있는 의료요원들을 최전선에 배치된 군인들에 비유하며 보너스 지급을 약속했다.     푸틴은 코로나19 환자들을 치료하는 의사들에게 주당 8만 루블(약 130만원), 중견 간호사에게는 5만 루블, 초급 간호사에겐 2만5천루블의 

  인덱스           날짜             신문사  \
0   1  2020.04.08.            연합뉴스   
0   2  2020.04.08.          한국경제TV   
0   3  2020.04.08.            연합뉴스   
0   4  2020.04.08.             뉴시스   
0   5  2020.04.08.            연합뉴스   
0   6  2020.04.08.            연합뉴스   
0   7  2020.04.08.             YTN   
0   8  2020.04.08.            연합뉴스   
0   9  2020.04.08.      MoneyToday   
0  10  2020.04.08.      MoneyToday   
0  11  2020.04.08.             YTN   
0  12  2020.04.08.            연합뉴스   
0  13  2020.04.08.             뉴시스   
0  14  2020.04.08.             YTN   
0  15  2020.04.08.     SBS & SBSi    
0  16  2020.04.08.             뉴시스   
0  17  2020.04.08.     SBS & SBSi    
0  18  2020.04.08.     SBS & SBSi    
0  19  2020.04.08.     SBS & SBSi    
0  20  2020.04.08.             뉴시스   
0  21  2020.04.08.     SBS & SBSi    
0  22  2020.04.08.     SBS & SBSi    
0  23  2020.04.08.            한국일보   
0  24  2020.04.08.     SBS & SBSi    
0  25  2020.04.08.             뉴스1   
0  26  2020.

In [75]:
print(df['인덱스'], df["내용"][:1])

0     1
0     2
0     3
0     4
0     5
0     6
0     7
0     8
0     9
0    10
0    11
0    12
0    13
0    14
0    15
0    16
0    17
0    18
0    19
0    20
0    21
0    22
0    23
0    24
0    25
0    26
0    27
0    28
0    29
0    30
0    31
0    32
0    33
0    34
0    35
0    36
0    37
0    38
0    39
0    40
0    41
0    42
0    43
0    44
0    45
0    46
0    47
0    48
0    49
0    50
0    51
0    52
0    53
0    54
0    55
Name: 인덱스, dtype: object 0    "모스크바 발병자 5천841명"…현지 전문가 "6월중순께나 수그러들 것"(모스크바=...
Name: 내용, dtype: object


### 데이터 삽입(INSERT)
* Cursor Object 가져오기 : cursor = db.cursor()
* SQL 실행하기 : cursor.execute(SQL)
* 실행 mysql 서버에 확정 반영하기 : db.commit()

### 데이터 조회(SELECT)
* Cursor Object 가져오기 : cursor = db.cursor()
* SQL 실행하기 : cursor.execute(SQL)
* mysql 서버로부터 데이터 가져오기 : fetch 메서드 사용
   * fetchall(): Fetch all the rows
   * fetchmany(size=None): Fetch several rows
   * fetchone(): Fetch the next row

In [16]:
import pymysql

# 접속
db = pymysql.connect(host='',
                     port=3306,
                     user='user',
                     passwd='',
                     db='mydb',
                     charset='utf8')

# 커서 가져오기
cursor = db.cursor()
# print (cursor)


# SQL 문 만들기
# sql = '''
   CREATE TABLE news(
       id INT UNSIGNED NOT NULL AUTO_INCREMENT,
       date VARCHAR(25) NOT NULL,
       company VARCHAR(15) NOT NULL,
       title VARCHAR(100) NOT NULL,
       content VARCHAR(3000) NOT NULL,
       PRIMARY KEY(id) 
);  
# '''

# cursor.execute(sql)

#실행하기
cursor.execute("desc news")
results = cursor.fetchall()

for result in results:
    print (result)
# # # DB에 Complete 하기
db.commit()

# # # DB 연결 닫기
db.close()



('id', 'int(10) unsigned', 'NO', 'PRI', None, 'auto_increment') <class 'tuple'>
('date', 'varchar(25)', 'NO', '', None, '') <class 'tuple'>
('company', 'varchar(15)', 'NO', '', None, '') <class 'tuple'>
('title', 'varchar(100)', 'NO', '', None, '') <class 'tuple'>
('content', 'varchar(3000)', 'NO', '', None, '') <class 'tuple'>
